# Curve Reconstruction

## CS 480 Computational Geometry, Spring 2020

### Dr. John C. Bowers

In this lab you will implement the CRUST algorithm discovered by N. Amenta, M. Bern, and D. Eppstein. This work builds on two of your previous labs.  

## Overview

Start by reading section 5.7. Your job is to implement the CRUST algorithm from the description in the book. Essentially this task amounts to understanding the algorithm as it is defined in the blue box at the end of the section, and using the components you have implemented in labs 5 and 6 to output the curves reconstructed by CRUST.

## Your Task

### Coding

This is going to be a little open ended because I want you to think about how to properly refactor your code and organize it. Your job is to implement the `crust` function found below the starter code, which takes as input a list of points and returns list of `SegmentE2` objects representing the crust reconstruction of the curve(s). The starter code an a graphical output to play with are below your code. Remember that testing via the graphics is a bad idea, since errors are not reported to the console when the occur in the graphics. Better to create dummy data and new code blocks directly and test your code on your own. 

### Questions (to answer after your code works and you've played with the graphical editor a bit):

1. Why does the `crust` function return a list of `SegmentE2` objects instead of a single `Polygon` object? 

    _Your answer here:_ 
    
2. Suppose you wanted to organize the curves that are output from the `crust` function as a list of `Polygon` objects. How would you go about doing this (give a high-level description with enough detail to understand the algorithm--doesn't have to be full pseudocode)? 

    _Your answer here:_ 

## Starter Code

As usual we will need some preliminary starter code. 

You have been given the usual primitives as well as implementations of `incrTriangulation` for triangulating a list of `PointE2` objects and `delaunify` for converting a triangulation of a point set into a Delaunay triangulation. 


In [ ]:
from koebe.geometries.euclidean2 import PointE2, SegmentE2

#
# PRIMITIVES
#

def areaOfParallelogram(A: PointE2, B: PointE2, C: PointE2) -> float:
    return (B.x - A.x) * (C.y - A.y) - (C.x - A.x) * (B.y - A.y)

def areaOfTriangle(A: PointE2, B: PointE2, C: PointE2) -> float:
    return 0.5 * areaOfParallelogram(A, B, C)

def leftHandTurn(A: PointE2, B: PointE2, C: PointE2) -> bool:
    return areaOfTriangle(A, B, C) > 0

def inCircle(A: PointE2, B: PointE2, C: PointE2, D: PointE2) -> bool:
    """
    Returns true if D is in the circle through points A, B, and C. 
    """
    adx = A.x - D.x
    ady = A.y - D.y
    bdx = B.x - D.x
    bdy = B.y - D.y
    cdx = C.x - D.x
    cdy = C.y - D.y

    abdet = adx * bdy - bdx * ady
    bcdet = bdx * cdy - cdx * bdy
    cadet = cdx * ady - adx * cdy
    alift = adx * adx + ady * ady
    blift = bdx * bdx + bdy * bdy
    clift = cdx * cdx + cdy * cdy
    
    return (alift * bcdet + blift * cadet + clift * abdet) > 0

def orient(p1, p2, p3):
    return (p1, p2, p3) if leftHandTurn(p1, p2, p3) else (p1, p3, p2)

#
# DATA STRUCTURES
#

#
# Doubly-connected edge list structure
# 

class DCEL:
    """
    The basic DCEL container. 
    
    Attributes:
        verts: List[Vertex] - The list of vertices. 
        halfEdges: List[HalfEdge] - The list of half-edges. 
        faces: List[Face] - The list of faces. 
        outerFace: Face - The outerFace (None if there isn't one.)
    """
    def __init__(self):
        self.verts = []
        self.halfEdges = []
        self.faces = []
        self.outerFace = None
    
class Vertex:
    """
    The basic DCEL Vertex object. 
    
    Attributes:
        dcel: DCEL - The parent DCEL. 
        outgoingHalfEdge: HalfEdge - Any one half-edge with this vertex as its origin.
        
    """
    def __init__(self, dcel, outgoingHalfEdge=None, pos=None):
        self.dcel = dcel
        self.dcel.verts.append(self)
        self.outgoingHalfEdge = outgoingHalfEdge
        self.pos = pos

class Face:
    def __init__(self, dcel, incidentHalfEdge=None):
        self.dcel = dcel
        self.dcel.faces.append(self)
        self.incidentHalfEdge = incidentHalfEdge
    
    def incidentHalfEdges(self):
        result = []
        curr = self.incidentHalfEdge
        while True:
            result.append(curr)
            curr = curr.next
            if curr == self.incidentHalfEdge:
                break
        return result
    
    def incidentVertices(self):
        return [he.origin for he in self.incidentHalfEdges()]

class HalfEdge:
    def __init__(self, 
                 dcel, 
                 origin=None, 
                 face=None, 
                 prev=None, 
                 next=None, 
                 twin=None):
        
        self.dcel   = dcel
        self.dcel.halfEdges.append(self)
        
        self.origin = origin
        self.face   = face
        self.prev   = prev
        self.next   = next
        self.twin   = twin
    
    def makeNext(self, he):
        self.next = he
        he.prev = self
    
    def makeTwin(self, he):
        self.twin = he
        he.twin = self
    
    @property # The @property annotation let's you "run" this as
              # he.destination instead of he.destination()
    def destination(self):
        return self.twin.origin
    
    def isLegal(self) -> bool:
        
        t0, t1 = self, self.twin
        
        a, b = t0.next, t0.prev
        c, d = t1.next, t1.prev
        
        A, B, C, D = a.origin.pos, b.origin.pos, c.origin.pos, d.origin.pos
        
        return (self.face == self.dcel.outerFace 
                or self.twin.face == self.dcel.outerFace
                or not (inCircle(A, B, C, D) or inCircle(C, D, A, B)))
    
    def flip(self):
        # First check if this is a valid flippable edge (i.e. an illegal edge)
        # Then do the flip by fixing self and self.twin's pointers (and the surrounding
        # half-edges of the flip quadrilateral). HINT: Remember that .face and .origin
        # pointers need to be updated for self and self.twin and .face pointers need
        # to be updated for all the half-edges incident the outer quadrilateral. 
        
        t0, t1 = self, self.twin
        f0, f1 = t0.face, t1.face
        
        a, b = t0.next, t0.prev
        c, d = t1.next, t1.prev
        
        A, B, C, D = a.origin, b.origin, c.origin, d.origin
        
        if not (leftHandTurn(A.pos, B.pos, C.pos) == leftHandTurn(B.pos, C.pos, D.pos)
            and leftHandTurn(B.pos, C.pos, D.pos) == leftHandTurn(C.pos, D.pos, A.pos)
            and leftHandTurn(C.pos, D.pos, A.pos) == leftHandTurn(D.pos, A.pos, B.pos)):
            return
        
        # Update vertex-halfEdge pointers
        t0.origin = D;
        t1.origin = B;
        C.outgoingHalfEdge = c
        A.outgoingHalfEdge = a
        
        # Update next/prev pointers
        t0.makeNext(b)
        b.makeNext(c)
        c.makeNext(t0)
        
        t1.makeNext(d)
        d.makeNext(a)
        a.makeNext(t1)
        
        # Update face pointers
        t0.face = b.face = c.face = f0; f0.incidentHalfEdge = t0
        t1.face = a.face = d.face = f1; f1.incidentHalfEdge = t1

def splitFace(e1: HalfEdge, e2: HalfEdge) -> None:
    if e1.face != e2.face:
        return
    
    newFace = Face(e1.dcel)

    s1 = HalfEdge(dcel=e1.dcel, origin=e2.origin, face=e1.face)
    s2 = HalfEdge(dcel=e1.dcel, origin=e1.origin, face=newFace)
    s1.makeTwin(s2)

    e2.prev.makeNext(s1)
    e1.prev.makeNext(s2)
    s1.makeNext(e1)
    s2.makeNext(e2)

    newFace.incidentHalfEdge = e2
    e1.face.incidentHalfEdge = e1

    for e in e1.face.incidentHalfEdges(): e.face = e1.face
    for e in newFace.incidentHalfEdges(): e.face = newFace
        
#
# ALGORITHMS
#

from typing import List

def triangleDCEL(p1: PointE2, p2: PointE2, p3: PointE2) -> DCEL:
    """
    Convenience method for creating a DCEL representing a triangle given its three corners. 
    """
    result = DCEL()
    result.outerFace = Face(result)
    
    tri = Face(result)
    
    A, B, C = [Vertex(dcel=result, pos=p) for p in orient(p1, p2, p3)]
    
    AB = HalfEdge(dcel=result, origin=A, face=tri)
    BC = HalfEdge(dcel=result, origin=B, face=tri)
    CA = HalfEdge(dcel=result, origin=C, face=tri)
    
    AC = HalfEdge(dcel=result, origin=A, face=result.outerFace)
    CB = HalfEdge(dcel=result, origin=C, face=result.outerFace)
    BA = HalfEdge(dcel=result, origin=B, face=result.outerFace)
    
    AB.makeTwin(BA)
    BC.makeTwin(CB)
    CA.makeTwin(AC)
    
    AB.makeNext(BC)
    BC.makeNext(CA)
    CA.makeNext(AB)
    
    AC.makeNext(CB)
    CB.makeNext(BA)
    BA.makeNext(AC)
    
    tri.incidentHalfEdge = AB
    result.outerFace.incidentHalfEdge = BA
    
    return result

def findLowerTangent(dcel, p):
    for he in dcel.outerFace.incidentHalfEdges():
        if (leftHandTurn(he.prev.origin.pos, he.origin.pos, p) 
            and leftHandTurn(he.next.origin.pos, he.origin.pos, p)):
            return he
    return None

def findUpperTangent(dcel, p):
    for he in dcel.outerFace.incidentHalfEdges():
        if (leftHandTurn(p, he.origin.pos, he.next.origin.pos) 
            and leftHandTurn(p, he.origin.pos, he.prev.origin.pos)):
            return he
    return None

def incrTriangulation(S: List[PointE2]) -> DCEL:
    thePoints = sorted(S, key=lambda p:(p.x, p.y))
    result = triangleDCEL(*thePoints[0:3])
    for p in thePoints[3:]:
        lowerTangent = findLowerTangent(result, p)
        upperTangent = findUpperTangent(result, p)
        
        upper_orig_prev = upperTangent.prev
        
        v = Vertex(dcel=result, pos=p)
        f = Face(dcel=result, incidentHalfEdge=upperTangent)

        upper_inner = HalfEdge(dcel=result, origin=v, face=f)
        lower_inner = HalfEdge(dcel=result, origin=lowerTangent.origin, face=f)
        
        upper_outer = HalfEdge(dcel=result, origin=upperTangent.origin, face=result.outerFace)
        lower_outer = HalfEdge(dcel=result, origin=v, face=result.outerFace)
        
        upper_inner.makeTwin(upper_outer)
        lower_inner.makeTwin(lower_outer)
        
        upper_inner.makeNext(upperTangent)
        lowerTangent.prev.makeNext(lower_inner)
        lower_inner.makeNext(upper_inner)
        
        upper_orig_prev.makeNext(upper_outer)
        upper_outer.makeNext(lower_outer)
        lower_outer.makeNext(lowerTangent)
        
        result.outerFace.incidentHalfEdge = upper_outer
        f.incidentHalfEdge = upperTangent
        v.outgoingHalfEdge = upper_inner
        
        for e in f.incidentHalfEdges(): e.face = f
        
        curr = upper_inner.next.next
        vHe = upper_inner
        while curr != lower_inner:
            curr = curr.next
            splitFace(curr.prev, vHe)
            vHe = curr.prev.prev
            
    return result
        
        
def delaunify(triangulation: DCEL):
    while True:
        illegalHEs = [he for he in triangulation.halfEdges if not he.isLegal()]
        if len(illegalHEs) == 0:
            return
        else: 
            illegalHEs[0].flip()
            
#
# GRAPHICS
#

from ipycanvas import Canvas, hold_canvas

def CrustCanvas(size, pts, draw_func = None, hIdx = -1):

    points = list(pts)
    canvas = Canvas(size=size)
    selectedPointIdx = -1
    
    segs = crust(points)
    
    def _canvas_draw():
        nonlocal canvas, points, selectedPointIdx, draw_func, segs, hIdx
        
        with hold_canvas(canvas):
            canvas.clear()

            if draw_func != None:
                draw_func(canvas, points)
    
            canvas.stroke_style = 'black'
            for seg in segs:
                u = seg.source
                v = seg.target
                canvas.begin_path()
                canvas.move_to(int(u.x), int(u.y))
                canvas.line_to(int(v.x), int(v.y))
                canvas.stroke()

            canvas.fill_style = 'blue'
            canvas.fill_rects([round(p.x) - 4 for p in points], 
                              [round(p.y) - 4 for p in points], 
                              8)
            
            if selectedPointIdx != -1:
                canvas.fill_style = 'red'
                canvas.fill_rect(round(points[selectedPointIdx].x) - 4, 
                                 round(points[selectedPointIdx].y) - 4, 
                                 8)
            
    def handle_mouse_down(x, y):
        nonlocal selectedPointIdx
        # See if any point is close to x, y
        cursorPoint = PointE2(x, y)
        sqDists = [p.distSqTo(cursorPoint) for p in points]
        minIdx = sqDists.index(min(sqDists))
        
        if sqDists[minIdx] < 24:
            selectedPointIdx = minIdx
        
        _canvas_draw()

    def handle_mouse_up(x, y):
        nonlocal selectedPointIdx, segs
        if selectedPointIdx >= 0:  
            segs = crust(points)
        selectedPointIdx = -1 # No point is selected anymore.
        _canvas_draw()

    def handle_mouse_move(x, y):
        nonlocal selectedPointIdx
        if selectedPointIdx >= 0:
            points[selectedPointIdx] = PointE2(x, y)
        _canvas_draw()
    
    canvas.on_mouse_down(handle_mouse_down)
    canvas.on_mouse_up(handle_mouse_up)
    canvas.on_mouse_move(handle_mouse_move)
    
    _canvas_draw()
    
    return canvas


# Your Part

In [ ]:
def crust(S: List[PointE2]) -> List[SegmentE2]:
    # Hints: 
    # 1. You can create a set from a list of points like this: set(S). You can then use the "in" keyword
    #    to test whether a point is in the set or not. You may want to do something like this to differentiate 
    #    between which vertices have a .pos point coming from the original set S and which ones come from the
    #    Voronoi diagram of S. 
    # 2. You can also attach information to a vertex after the DCEL is created. For example, you can loop
    #    over all of the vertices by index and set a .idx field on each vertex. 
    # 3. If you do step 2, then you can make sure to only output one segment per edge by dealing only with half
    #    edges he where he.origin.idx < he.destination.idx (it's twin will have he.destination.idx < he.origin.idx). 
    #    Or you can use some sort of marking scheme where whenever you output a half edge as a SegmentE2, you 
    #    set .marked = True and .twin.marked = True (you will have to initialize these, of course) and then only
    #    output when .marked = False. 
    
    # TODO Replace this dummy code: 
    return [SegmentE2(S[i-1], S[i]) for i in range(len(S))]

In [ ]:
CrustCanvas(
    size=(800, 800),
    pts=[
        PointE2(p.x * 400 + 200, p.y * 400 + 200)
        for p in [PointE2(x=0.3499044588330262, y=0.3797005286201811),
             PointE2(x=0.5201333339235448, y=0.9300817569402503),
             PointE2(x=0.8494277939564803, y=0.7097144311136273),
             PointE2(x=0.09988849675657077, y=0.500098700684348),
             PointE2(x=0.1195031280104291, y=0.6108006260172161),
             PointE2(x=0.6006137689383454, y=0.3999995104449736),
             PointE2(x=0.3397068685895621, y=0.9306493473567267),
             PointE2(x=0.5606174873265656, y=0.20905817180221475),
             PointE2(x=0.9303695813917385, y=0.7798831066352676),
             PointE2(x=0.8500959745435225, y=0.66919193472458),
             PointE2(x=0.29082705806577497, y=0.1705913941577888),
             PointE2(x=0.6295744409018594, y=0.9301070037604309),
             PointE2(x=0.24016093731015287, y=0.14071736962904102),
             PointE2(x=0.26089318680599644, y=0.7899808442224125),
             PointE2(x=0.2006793404747653, y=0.7603176385872952),
             PointE2(x=0.19979891142916784, y=0.2201740532782679),
             PointE2(x=0.16007006560255516, y=0.7097523042148572),
             PointE2(x=0.45017892696614337, y=0.2796607927729292),
             PointE2(x=0.8997656088782054, y=0.8396800568327721),
             PointE2(x=0.12986030572260215, y=0.5195729296508476),
             PointE2(x=0.09023610137157555, y=0.4500134554435722),
             PointE2(x=0.24014318753126646, y=0.9206161487361288),
             PointE2(x=0.8202800578677243, y=0.880059847178049),
             PointE2(x=0.13030559015835178, y=0.6608190447940824),
             PointE2(x=0.44979549128546814, y=0.9303066756067251),
             PointE2(x=0.8705446070919329, y=0.8698145698287529),
             PointE2(x=0.7093838311623829, y=0.9195081709309197),
             PointE2(x=0.2698199872417973, y=0.8498965278556363),
             PointE2(x=0.5196394356247528, y=0.4101510570050677),
             PointE2(x=0.3897783417759427, y=0.9304237405318859),
             PointE2(x=0.2708615389541595, y=0.8002991479560169),
             PointE2(x=0.5300064705778578, y=0.1098829143769603),
             PointE2(x=0.7297721826248464, y=0.43022287352524435),
             PointE2(x=0.11021288705020157, y=0.35071731288206287),
             PointE2(x=0.3003204578239861, y=0.8004146360051164),
             PointE2(x=0.19967961351724553, y=0.17998963960043224),
             PointE2(x=0.8901847357485659, y=0.7107199607450553),
             PointE2(x=0.289740726024328, y=0.23971494486524036),
             PointE2(x=0.30007283132433404, y=0.20037393812643012),
             PointE2(x=0.29008021449112686, y=0.2504756021391894),
             PointE2(x=0.30050340817949567, y=0.25046414843760945),
             PointE2(x=0.2001877096190833, y=0.27012090348574735),
             PointE2(x=0.34002436647958995, y=0.3400769799223615),
             PointE2(x=0.2892577417011757, y=0.8206348514795312),
             PointE2(x=0.3198308725143603, y=0.23029364854674805),
             PointE2(x=0.830403105190022, y=0.5708795591855115),
             PointE2(x=0.41023180888126276, y=0.16055126976834566),
             PointE2(x=0.12971396217138434, y=0.2797184027451086),
             PointE2(x=0.16996532069168446, y=0.280335989893403),
             PointE2(x=0.09990419055125564, y=0.3105898563927021),
             PointE2(x=0.5300394362571527, y=0.23953921017992907),
             PointE2(x=0.6593569096950701, y=0.39979318519452145),
             PointE2(x=0.36983526548768536, y=0.4099583102011466),
             PointE2(x=0.29009242083079906, y=0.25914731000385205),
             PointE2(x=0.2802589581432746, y=0.14985350772693395),
             PointE2(x=0.3503882076864453, y=0.2000500033456979),
             PointE2(x=0.4898774792716873, y=0.11022009547318712),
             PointE2(x=0.7793532540775527, y=0.4796205348015995),
             PointE2(x=0.2901149775243338, y=0.9306866948414034),
             PointE2(x=0.22985750596807103, y=0.8897331983860828),
             PointE2(x=0.12951965291685882, y=0.5504202563246784),
             PointE2(x=0.4396717308078143, y=0.4292841674803205),
             PointE2(x=0.7506773756812586, y=0.8902759363087519),
             PointE2(x=0.10039161501042379, y=0.40027264664673057),
             PointE2(x=0.9204252147024274, y=0.729623951507877)]
    ]
)

In [ ]:
# Here is a point set to test your code with: 

pts = [PointE2(x=0.3500002575685883, y=0.6198571715057889),
 PointE2(x=0.5199747419797536, y=0.0696655603527525),
 PointE2(x=0.8496981453666755, y=0.29030161845169505),
 PointE2(x=0.09980116472589984, y=0.5001276872906524),
 PointE2(x=0.11972190617692494, y=0.3896849585860642),
 PointE2(x=0.600381254276391, y=0.6003493171235834),
 PointE2(x=0.3395064411756996, y=0.06980745557065232),
 PointE2(x=0.560346532265904, y=0.7904916456622951),
 PointE2(x=0.9300920291542859, y=0.21977840465672216),
 PointE2(x=0.8497004295605517, y=0.33049376887961607),
 PointE2(x=0.29049384341369633, y=0.8298741576527823),
 PointE2(x=0.6299230437766031, y=0.06976488761639),
 PointE2(x=0.24043390363542697, y=0.859637892471355),
 PointE2(x=0.26039422446988714, y=0.2096059585287478),
 PointE2(x=0.20049479053646677, y=0.24017548116454968),
 PointE2(x=0.2002391428126469, y=0.7795421267972592),
 PointE2(x=0.15988456705893667, y=0.2898212316371148),
 PointE2(x=0.4504388179833366, y=0.7202206198378237),
 PointE2(x=0.8997931866822885, y=0.16001939474974983),
 PointE2(x=0.13001592053815308, y=0.4799509544183471),
 PointE2(x=0.08981121276931799, y=0.5502670475884855),
 PointE2(x=0.2403116770725899, y=0.07970761691991086),
 PointE2(x=0.8204884953375329, y=0.12019217450110936),
 PointE2(x=0.1304877840137006, y=0.339624716865601),
 PointE2(x=0.449914352507657, y=0.07001011955190789),
 PointE2(x=0.870435138476846, y=0.1299843348634018),
 PointE2(x=0.7096621114252744, y=0.08032420138817227),
 PointE2(x=0.27026772457793424, y=0.1498122521840106),
 PointE2(x=0.519693429517224, y=0.5898579148483493),
 PointE2(x=0.38977698411721917, y=0.06960204797685507),
 PointE2(x=0.27038927175327077, y=0.1996646401948866),
 PointE2(x=0.5299906062644556, y=0.8903841584066918),
 PointE2(x=0.7295409788671593, y=0.5696858154534449),
 PointE2(x=0.1098578231826783, y=0.6496281058277801),
 PointE2(x=0.30034906341813383, y=0.1996597915253984),
 PointE2(x=0.19956002586905314, y=0.8202377618002699),
 PointE2(x=0.8903174639252378, y=0.28977431047707397),
 PointE2(x=0.2897635386280603, y=0.7600959157906851),
 PointE2(x=0.2996823939362374, y=0.7998577097563419),
 PointE2(x=0.2898088600176013, y=0.7498724094301812),
 PointE2(x=0.30045775878120323, y=0.749806013981873),
 PointE2(x=0.2004899413154573, y=0.729602054439588),
 PointE2(x=0.34049503004177906, y=0.6603599175584155),
 PointE2(x=0.28958178314089883, y=0.17978549055774434),
 PointE2(x=0.31970169775531887, y=0.7698930902532759),
 PointE2(x=0.830309021197746, y=0.42955744130160056),
 PointE2(x=0.4104914420784699, y=0.8395282061787219),
 PointE2(x=0.1298224932901229, y=0.7199087634653544),
 PointE2(x=0.1700553081742327, y=0.7196130330353688),
 PointE2(x=0.09955860721001374, y=0.6895175622184588),
 PointE2(x=0.5298626145255316, y=0.7603707391023369),
 PointE2(x=0.6597738526439552, y=0.6004947559944046),
 PointE2(x=0.37001916052410244, y=0.5904156984805515),
 PointE2(x=0.28974127815034856, y=0.7403930883302955),
 PointE2(x=0.2798898919032832, y=0.8497876674730808),
 PointE2(x=0.3504322536040336, y=0.8001524558473206),
 PointE2(x=0.48955214277932574, y=0.8898925107767962),
 PointE2(x=0.7795438357952563, y=0.5203708609600758),
 PointE2(x=0.28964774538062954, y=0.06950856831533411),
 PointE2(x=0.22956341998439528, y=0.11009641525077776),
 PointE2(x=0.1297846673148995, y=0.4499256223525013),
 PointE2(x=0.4398129909652824, y=0.5703549992238507),
 PointE2(x=0.7502195678126496, y=0.11013660010969897),
 PointE2(x=0.10046186725717977, y=0.6001994348256868),
 PointE2(x=0.920296635240604, y=0.2703537520832185)]

crust(pts)